In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

2024-10-05 14:45:37.278093: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-10-05 14:45:37.287745: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-10-05 14:45:37.298863: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:8454] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-10-05 14:45:37.302716: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1452] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-10-05 14:45:37.311072: I tensorflow/core/platform/cpu_feature_guar

In [2]:
# Load data
from tensorflow.keras.datasets import fashion_mnist

(x_train_set, y_train_set), (x_test, y_test) = fashion_mnist.load_data()

# Split data
from sklearn.model_selection import train_test_split

x_train, x_valid, y_train, y_valid = train_test_split(
    x_train_set, y_train_set, random_state=1
)

# Preprocessing
x_train = x_train / 255.0
x_valid = x_valid / 255.0
x_test = x_test / 255.0

In [3]:
# 不用執行，只是預先確認分割的資料結果
print(y_train[:10])  # 全部
ind_B = (y_train == 4) | (y_train == 5)  # 篩選出原分類：4涼鞋，5外套
print(ind_B[:10])
# 找出ind_B=True,且y_train=5的轉成 1.0
# 若y_train=4則會變成 0.0
y_B = (y_train[ind_B] == 5).astype(np.float32)
print(y_B[:10])
y_A = y_train[~ind_B]  # 排除 ind_B，即找出ind_B=False
print(y_A[:10])
y_A[y_A > 5] -= 2  # 將原分類結果 > 5的，改成-2
print(y_A[:10])

[0 9 8 2 2 5 1 4 1 1]
[False False False False False  True False  True False False]
[1. 0. 0. 0. 1. 1. 1. 0. 0. 0.]
[0 9 8 2 2 1 1 1 8 9]
[0 7 6 2 2 1 1 1 6 7]


In [4]:
# 將 data拆成 2部分
def split_data(x, y):
    ind_B = (y == 4) | (y == 5)  # 篩選原分類：4涼鞋，5外套
    y_B = (y[ind_B] == 5).astype(np.float32)  # 原分類 5 轉成 1；分類 4轉成 0
    y_A = y[~ind_B]  # 排除 ind_B
    y_A[y_A > 5] -= 2  # 原分類編號>5的，-2
    return (x[~ind_B], y_A), (x[ind_B], y_B)


(x_train_A, y_train_A), (x_train_B, y_train_B) = split_data(x_train, y_train)
(x_valid_A, y_valid_A), (x_valid_B, y_valid_B) = split_data(x_valid, y_valid)
(x_test_A, y_test_A), (x_test_B, y_test_B) = split_data(x_test, y_test)

x_train_B, y_train_B = x_train_B[:100], y_train_B[:100]

In [5]:
x_train_A.shape, y_train_A[:10]

((35968, 28, 28), array([0, 7, 6, 2, 2, 1, 1, 1, 6, 7], dtype=uint8))

In [6]:
x_train_B.shape, y_train_B[:10]

((100, 28, 28), array([1., 0., 0., 0., 1., 1., 1., 0., 0., 0.], dtype=float32))

### Model A

In [7]:
model_A = keras.models.Sequential()
# layer 0
model_A.add(keras.layers.Flatten(input_shape=[28, 28]))
# layer 1-4
for hidden_i in (200, 150, 100, 50):
    model_A.add(keras.layers.Dense(hidden_i, activation="relu"))
# layer5:輸出 8 類
model_A.add(keras.layers.Dense(8, activation="softmax"))

/home/chesterxalan/github/Classwork/Classwork-PythonDLApplicationDevelopment/.venv/lib/python3.12/site-packages/keras/src/layers/reshaping/flatten.py:37: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
I0000 00:00:1728110809.609875   31512 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728110809.648014   31512 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728110809.648068   31512 cuda_executor.cc:1001] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
I0000 00:00:1728110809.651319   31512 c

In [8]:
model_A.summary()


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       157,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        15,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           408 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 207,708 (811.36 KB)

 Trainable params: 207,708 (811.36 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
model_A.compile(
    loss="sparse_categorical_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=["accuracy"],
)

In [10]:
train = model_A.fit(
    x_train_A, y_train_A, epochs=20, validation_data=(x_valid_A, y_valid_A)
)

Epoch 1/20


I0000 00:00:1728110824.310312   32758 service.cc:146] XLA service 0x7f9b1c006210 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1728110824.310357   32758 service.cc:154]   StreamExecutor device (0): NVIDIA GeForce RTX 4060 Laptop GPU, Compute Capability 8.9
2024-10-05 14:47:04.339856: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:268] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
2024-10-05 14:47:04.402448: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:531] Loaded cuDNN version 8907
2024-10-05 14:47:07.631177: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_237', 28 bytes spill stores, 28 bytes spill loads

2024-10-05 14:47:07.702769: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory

  82/1124 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.1417 - loss: 2.0977

I0000 00:00:1728110828.854397   32758 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.


1124/1124 ━━━━━━━━━━━━━━━━━━━━ 8s 3ms/step - accuracy: 0.3204 - loss: 1.9473 - val_accuracy: 0.6907 - val_loss: 1.3127
Epoch 2/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.7384 - loss: 1.0986 - val_accuracy: 0.7875 - val_loss: 0.6855
Epoch 3/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.7965 - loss: 0.6457 - val_accuracy: 0.8065 - val_loss: 0.5651
Epoch 4/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8115 - loss: 0.5514 - val_accuracy: 0.8230 - val_loss: 0.5177
Epoch 5/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8248 - loss: 0.5081 - val_accuracy: 0.8276 - val_loss: 0.4906
Epoch 6/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8297 - loss: 0.4837 - val_accuracy: 0.8294 - val_loss: 0.4722
Epoch 7/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - accuracy: 0.8389 - loss: 0.4630 - val_accuracy: 0.8365 - val_loss: 0.4558
Epoch 8/20
1124/1124 ━━━━━━━━━━━━━━━━━━━━ 2s 2ms/step - accuracy: 0.8411 - loss: 0.4560 - val_accurac

In [11]:
model_A.evaluate(x_test_A, y_test_A)

250/250 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8708 - loss: 0.3820


[0.38873663544654846, 0.862375020980835]

In [12]:
model_A.save("model_A.h5")

### Model B

In [13]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))

for hidden_i in (300, 150, 100, 80):
    model_B.add(keras.layers.Dense(hidden_i, activation="relu"))

model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [14]:
model_B.summary()

Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten_1 (Flatten)             │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 300)            │       235,500 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 150)            │        45,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 100)            │        15,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_8 (Dense)                 │ (None, 80)             │         8,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_9 (Dense)                 │ (None, 1)              │            81 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 303,911 (1.16 MB)

 Trainable params: 303,911 (1.16 MB)

 Non-trainable params: 0 (0.00 B)

In [15]:
model_B.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=["accuracy"],
)

In [16]:
train = model_B.fit(
    x_train_B, y_train_B, epochs=20, validation_data=(x_valid_B, y_valid_B)
)

Epoch 1/20


2024-10-05 14:54:57.222530: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_342', 8 bytes spill stores, 8 bytes spill loads

2024-10-05 14:54:57.261010: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_338', 28 bytes spill stores, 28 bytes spill loads

2024-10-05 14:54:57.448029: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_338', 28 bytes spill stores, 28 bytes spill loads

2024-10-05 14:54:57.620714: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_338', 20 bytes spill stores, 20 bytes spill loads



4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 913ms/step - accuracy: 0.1798 - loss: 0.7321

2024-10-05 14:55:03.182019: I external/local_xla/xla/stream_executor/cuda/cuda_asm_compiler.cc:393] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_31', 16 bytes spill stores, 16 bytes spill loads



4/4 ━━━━━━━━━━━━━━━━━━━━ 10s 2s/step - accuracy: 0.1918 - loss: 0.7308 - val_accuracy: 0.3251 - val_loss: 0.7178
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 60ms/step - accuracy: 0.3417 - loss: 0.7172 - val_accuracy: 0.3713 - val_loss: 0.7107
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 57ms/step - accuracy: 0.4029 - loss: 0.7091 - val_accuracy: 0.4040 - val_loss: 0.7061
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.3864 - loss: 0.7050 - val_accuracy: 0.4417 - val_loss: 0.7005
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.4222 - loss: 0.7001 - val_accuracy: 0.4936 - val_loss: 0.6933
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.4385 - loss: 0.6915 - val_accuracy: 0.5266 - val_loss: 0.6869
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 61ms/step - accuracy: 0.4637 - loss: 0.6901 - val_accuracy: 0.5418 - val_loss: 0.6804
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 62ms/step - accuracy: 0.5019 - loss: 0.6817 - val_accuracy: 0.5505 - val_loss: 0.6762
Epoch 9/20
4

In [17]:
model_B.evaluate(x_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 39ms/step - accuracy: 0.5742 - loss: 0.6196


[0.624629557132721, 0.5584999918937683]

### 載入模型

In [26]:
# Model_tr (Transfer Learning)
# 1. Load model_A 載入模型A
model_A = keras.models.load_model("model_A.h5")

In [19]:
model_A.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       157,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        15,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 50)             │         5,050 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 8)              │           408 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 207,710 (811.37 KB)

 Trainable params: 207,708 (811.36 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [41]:
# 2. Reuse Lower Layers 使用較低層
# flatten (Flatten) / dense (Dense) / dense_1 (Dense) / dense_2 (Dense)
model_tr = keras.models.Sequential(model_A.layers[:-2])  # 不含後二層

In [42]:
model_tr.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       157,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        15,100 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,250 (790.04 KB)

 Trainable params: 202,250 (790.04 KB)

 Non-trainable params: 0 (0.00 B)

In [43]:
# 3. Replace Upper Layers
model_tr.add(keras.layers.Dense(80, activation="relu"))
model_tr.add(keras.layers.Dense(50, activation='relu'))
model_tr.add(keras.layers.Dense(1, activation="sigmoid"))

In [44]:
model_tr.summary()

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ flatten (Flatten)               │ (None, 784)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 200)            │       157,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 150)            │        30,150 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 100)            │        15,100 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_18 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_19 (Dense)                │ ?                      │   0 (unbuilt) │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_20 (Dense)                │ ?                      │   0 (unbuilt) │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 202,250 (790.04 KB)

 Trainable params: 202,250 (790.04 KB)

 Non-trainable params: 0 (0.00 B)

In [45]:
# 4. Train in the First Few Epochs 初期訓練
for layer in model_tr.layers[:-3]:
    layer.trainable = False  # 鎖定不做訓練

In [46]:
model_tr.summary(show_trainable=True)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ flatten (Flatten)           │ (None, 784)           │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 200)           │    157,000 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_1 (Dense)             │ (None, 150)           │     30,150 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_2 (Dense)             │ (None, 100)           │     15,100 │   N   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_18 (Dense)            │ ?                     │          0 │   -   │
│                             │                       │  (unbuilt) │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_19 (Dense)            │ ?                     │          0 │   -   │
│                             │                       │  (unbuilt) │       │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_20 (Dense)            │ ?                     │          0 │   -   │
│                             │                       │  (unbuilt) │       │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 202,250 (790.04 KB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 202,250 (790.04 KB)

In [47]:
model_tr.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=["accuracy"],
)

In [48]:
train = model_tr.fit(
    x_train_B, y_train_B, epochs=10, validation_data=(x_valid_B, y_valid_B)
)

Epoch 1/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 906ms/step - accuracy: 0.5019 - loss: 1.5939 - val_accuracy: 0.5067 - val_loss: 1.2024
Epoch 2/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.5403 - loss: 1.1331 - val_accuracy: 0.5232 - val_loss: 0.8907
Epoch 3/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.5504 - loss: 0.8764 - val_accuracy: 0.6503 - val_loss: 0.6775
Epoch 4/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step - accuracy: 0.6432 - loss: 0.6729 - val_accuracy: 0.8211 - val_loss: 0.5887
Epoch 5/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.8245 - loss: 0.5845 - val_accuracy: 0.8757 - val_loss: 0.5389
Epoch 6/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8913 - loss: 0.5360 - val_accuracy: 0.8534 - val_loss: 0.5061
Epoch 7/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step - accuracy: 0.8987 - loss: 0.4943 - val_accuracy: 0.8184 - val_loss: 0.4812
Epoch 8/10
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.8426 - loss: 0.4836 - val_accuracy: 0.8191 - val_loss: 0.4649

In [49]:
# 5. Continue Training后期訓練(全部參數都要參與訓練)
for layer in model_tr.layers[:-3]:
    layer.trainable = True  # 將鎖定的層打開，參與訓練

In [50]:
model_tr.summary(show_trainable=True)

Model: "sequential_4"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━┓
┃ Layer (type)                ┃ Output Shape          ┃    Param # ┃ Trai… ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━┩
│ flatten (Flatten)           │ (None, 784)           │          0 │   -   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense (Dense)               │ (None, 200)           │    157,000 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_1 (Dense)             │ (None, 150)           │     30,150 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_2 (Dense)             │ (None, 100)           │     15,100 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_18 (Dense)            │ (None, 80)            │      8,080 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_19 (Dense)            │ (None, 50)            │      4,050 │   Y   │
├─────────────────────────────┼───────────────────────┼────────────┼───────┤
│ dense_20 (Dense)            │ (None, 1)             │         51 │   Y   │
└─────────────────────────────┴───────────────────────┴────────────┴───────┘

 Total params: 214,433 (837.63 KB)

 Trainable params: 214,431 (837.62 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 2 (12.00 B)

In [51]:
model_tr.compile(
    loss="binary_crossentropy",
    optimizer=keras.optimizers.SGD(learning_rate=1e-3),
    metrics=["accuracy"],
)

In [52]:
train = model_tr.fit(
    x_train_B, y_train_B, epochs=20, validation_data=(x_valid_B, y_valid_B)
)

Epoch 1/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 4s 806ms/step - accuracy: 0.9171 - loss: 0.4304 - val_accuracy: 0.8811 - val_loss: 0.4177
Epoch 2/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 69ms/step - accuracy: 0.9076 - loss: 0.4148 - val_accuracy: 0.9053 - val_loss: 0.4002
Epoch 3/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 58ms/step - accuracy: 0.9197 - loss: 0.4059 - val_accuracy: 0.9225 - val_loss: 0.3846
Epoch 4/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.9375 - loss: 0.3775 - val_accuracy: 0.9350 - val_loss: 0.3713
Epoch 5/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step - accuracy: 0.9592 - loss: 0.3620 - val_accuracy: 0.9562 - val_loss: 0.3566
Epoch 6/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 52ms/step - accuracy: 0.9663 - loss: 0.3319 - val_accuracy: 0.9616 - val_loss: 0.3442
Epoch 7/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 51ms/step - accuracy: 0.9753 - loss: 0.3330 - val_accuracy: 0.9707 - val_loss: 0.3313
Epoch 8/20
4/4 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step - accuracy: 0.9816 - loss: 0.3187 - val_accuracy: 0.9741 - val_loss: 0.3200

In [53]:
model_tr.evaluate(x_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 2s 27ms/step - accuracy: 0.9904 - loss: 0.2198


[0.22275663912296295, 0.9929999709129333]

In [54]:
model_B.evaluate(x_test_B, y_test_B)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.5742 - loss: 0.6196


[0.624629557132721, 0.5584999918937683]